In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import geopy
from geopy.distance import geodesic
from shapely.geometry import Point
import branca.colormap as cm
from ipywidgets import interact_manual, widgets, VBox, HBox
import ipywidgets as widgets
from sklearn.preprocessing import MinMaxScaler
import zipfile
import os
import fiona
pd.set_option('display.max_columns', None)


# Função de Leitura KMZ

In [ ]:
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

def read_kmz_directly(kmz_file):    # Descompactar o KMZ para uma pasta temporária
    output_folder = "kmz_temp"
    with zipfile.ZipFile(kmz_file, 'r') as kmz:
        kmz.extractall(output_folder)  # Extrair tudo para a pasta
    
    # Verificar os arquivos extraídos e localizar o arquivo KML
    extracted_files = os.listdir(output_folder)
    print(f"Arquivos extraídos: {extracted_files}")
    
    # Localizar o arquivo KML (geralmente chamado 'doc.kml' ou similar)
    kml_file = next((f for f in extracted_files if f.endswith('.kml')), None)
    if not kml_file:
        raise ValueError("Nenhum arquivo KML encontrado dentro do KMZ.")
    
    # Caminho completo para o arquivo KML
    kml_file_path = os.path.join(output_folder, kml_file)
    
    # Ler o KML com GeoPandas
    gdf = gpd.read_file(kml_file_path)
    return gdf
    

# Funções de Estações

In [ ]:
## Prepara o dataframe de estações para as outras análises
## Retorna o dataframe tratado
def tratar_df_estacoes(df_estacoes):
    ## Cria uma geometria "Point"
    df_estacoes = gpd.GeoDataFrame(df_estacoes, geometry = gpd.points_from_xy(df_estacoes['lon'], df_estacoes['lat']))
    ## Define crs
    df_estacoes.set_crs("EPSG:4326", inplace = True)
    return df_estacoes

## Cria circles (geometry Polygon) em volta de cada estação (geometry Point)
## Retorna o dataframe com a geometria "circles"
def criar_circulos_estacao(data, radius_km):
    circles = []
    for _, row in data.iterrows():
        circle_points = []
        center = (row['lat'], row['lon'])
        ## Dizemos que é aproximadamente um círculo, pois é um polígono regular com 360 vértices
        for angle in range(0,360, 1): 
            destination = geopy.distance.distance(kilometers = radius_km).destination(center, angle)
            circle_points.append(Point(destination.longitude, destination.latitude))
        circle = gpd.GeoSeries(circle_points).unary_union.convex_hull
        circles.append(circle)
    data.rename(columns = {'geometry':'point'}, inplace = True)
    circles_gdf = gpd.GeoDataFrame(data, geometry = circles)
    circles_gdf.rename(columns = {'geometry':'circle'}, inplace = True)
    return circles_gdf

## Adiciona os círculos no mapa, usa como base a função "criar_circulos_estacao"
## Retorna o Mapa com os círculos (geometry polygon) e pontos (geometry point) adicionados
def adicionar_estacoes_com_circulos_mapa(mapa, df_estacoes, radius_km):
    ## Definição dos layers para controle do usuário por camadas
    layer_estacoes =  folium.FeatureGroup(name = "Estações")
    layer_circulos_estacoes = folium.FeatureGroup(name = f"Redondezas {radius_km} km") 
    ## Se o df_estacoes já tem os circles, não precisa criá-los
    if "circle" not in df_estacoes:
        data = criar_circulos_estacao(df_estacoes, radius_km)
    else:
        data = df_estacoes.copy()
    ## Define "circle" como geometria a ser usada, pois há mais de uma em uso
    data.set_geometry("circle", inplace = True)
    ## Adiciona Circlemarkers ao layer
    for _, row in data.iterrows():
        folium.CircleMarker(
                    location=[row['lat'],row['lon']],
                    color = "blue",
                    radius= 5,
                    tooltip= row['name'],
                    fill=True,
                    fill_opacity=1,
                    fill_color="blue",
                ).add_to(layer_estacoes)
    ## Adiciona os Circles ao layer
    for _, row in data.iterrows():
        folium.GeoJson(
            row['circle'].__geo_interface__, 
            tooltip = row['name'],
            style_function=lambda x:{'fillColor': 'gray', 'color': 'gray', 'fillOpacity': 0.0},
        ).add_to(layer_circulos_estacoes)
        layer_estacoes.add_to(mapa)
        layer_circulos_estacoes.add_to(mapa)
        folium.LayerControl().add_to(mapa)
    return mapa

# Funções de Mapa

In [ ]:
## Define cores para mapas
def get_color(valor):
    if valor == 1:
        return "yellow"
    elif valor == 2:
        return "orange"
    elif valor == 3:
        return "red"
    elif valor == 4:
        return "darkred"
    elif valor == 5:
        return "blue"
    elif valor == 6:
        return "darkblue"
    elif valor == 7:
        return "purple"
    elif valor == 8:
        return "pink"
    elif valor == 9:
        return "green"
    elif valor == 10:
        return "darkgreen"
    elif valor == 11:
        return "lightblue"
    elif valor == 12:
        return "cyan"
    elif valor == 13:
        return "brown"
    elif valor == 14:
        return "gray"
    elif valor == 15:
        return "black"
    elif valor == 16:
        return "lightgray"
    elif valor == 17:
        return "white"
    else:
        return "black"  # Valor inválido

## Cria um mapa centrado em São Paulo
def criar_mapa_sp():
    mapa_sp = folium.Map(location = [-23.550520, -46.633308], zoom_start = 12)
    return mapa_sp

## Plota Circlemarkers de 1 cor, dado 1 df com Lat e Lon
def plot_circlemarkers_one_color(data, color, radius, name, pontuacao, layer):
    for _, row in data.iterrows():
        folium.CircleMarker(
                location = [row['lat'], row['lon']],
                color = color, 
                radius = radius,
                tooltip = row[name] + " " + str(row[pontuacao]), 
                popup = row[name] + " " + str(row[pontuacao]), 
                fill = True,
                fill_opacity = 1,
                fill_color = color,
        ).add_to(layer)

## Plota Circlemarkers de várias cores, dado 1 df com Lat e Lon e um Colormap branca
def plot_circlemarkers_multi_color(data, radius, name, pontuacao, layer, colormap):
    for _, row in data.iterrows():
        folium.CircleMarker(
                location = [row['lat'], row['lon']],
                color = colormap(row[pontuacao]), 
                radius = radius,
                tooltip = row[name] + " " + str(row[pontuacao]), 
                popup =  row[name] + " " + str(row[pontuacao]), 
                fill = True,
                fill_opacity = 1,
                fill_color = colormap(row[pontuacao]),
        ).add_to(layer)
        
    


## Funções de Outliers

In [ ]:
## Plota os Circlemarkers outliers dado um out_lower, out_upper e non_outliers, separados pela função "identificar_outliers"
def plot_circlemarkers_outliers(out_lower, out_upper, non_outliers, color_lower, color_upper, radius, 
                                name, pontuacao, layer, colormap):
    plot_circlemarkers_one_color(out_lower, color_lower, radius, name, pontuacao, layer)
    plot_circlemarkers_one_color(out_upper, color_upper, radius, name, pontuacao, layer)
    plot_circlemarkers_multi_color(non_outliers, radius, name, pontuacao, layer, colormap)

def identificar_outliers(df, coluna):
    """
    Identifica outliers em uma coluna de um DataFrame usando o método IQR.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os dados.
        coluna (str): Nome da coluna para identificar os outliers.

    Retorna:
        outliers_lower (pd.DataFrame): DataFrame contendo os outliers inferiores.
        outliers_upper (pd.DataFrame): DataFrame contendo os outliers superiores.
        non_outliers (pd.DataFrame): DataFrame contendo os dados não-outliers.
    """
    Q1 = df[coluna].quantile(0.25) 
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    lowerbound = Q1 - 1.5 * IQR
    upperbound = Q3 + 1.5 * IQR

    outliers_lower = df[df[coluna] < lowerbound]
    outliers_upper = df[df[coluna] > upperbound]
    non_outliers = df[(df[coluna] >= lowerbound) & (df[coluna] <= upperbound)]

    return outliers_lower, outliers_upper, non_outliers 


# Funções de Uso do Solo

In [ ]:
## Prepara o dataframe de ocorrencias para as outras análises
## Retorna o dataframe tratado
def tratar_df_uso_do_solo(df_uso_do_solo):
    uso_do_solo = df_uso_do_solo.copy()
    substituicoes = {
        "Residencial Horizontal Médio Padrão": 1,
        "Residencial Horizontal Baixo Padrão": 2,
        "Comércio e Serviço Horizontal": 3,
        "Residencial Vertical Médio Padrão": 4,
        "Residencial Horizontal Alto Padrão": 5,
        "Residencial Vertical Alto Padrão": 6,
        "Comércio e Serviço Vertical": 7,
        "Residencial Vertical Baixo Padrão": 8,
        "Terrenos Vagos": 9,
        "Indústrias": 10,
        "Escolas": 11,
        "Usos Coletivos": 12,
        "Usos Especiais": 13,
        "Armazéns e Depósitos": 14,
        "Sem correspondência": 15,
        "Outros Usos": 16,
        "Garagens": 17
    }

    uso_do_solo['uso_pred_simples_num'] = uso_do_solo['uso_pred_simples'].replace(substituicoes)
    uso_do_solo = uso_do_solo[['Name', 'uso_pred_simples','uso_pred_simples_num', 'uso_pred_60','qt_area_quadra','qt_area_construida', 'geometry']]
    # uso_do_solo = uso_do_solo.set_crs("EPSG:31983")  # Define o CRS, se ainda não estiver definido
    uso_do_solo = uso_do_solo.to_crs("EPSG:4326")
    return uso_do_solo
    
def analise_uso_do_solo(df_uso_do_solo, df_estacoes, radius_km, uso_pred_simples):
    mapa = criar_mapa_sp()
    df_estacoes_circulo = criar_circulos_estacao(df_estacoes, radius_km)
    df_estacoes_circulo.set_geometry("circle", inplace=True)  

    df_estacoes_circulo.set_crs("EPSG:4326", inplace = True)
    ## Quais ocorrencias eu vou considerar?
    df_uso_do_solo_filtrado = df_uso_do_solo[df_uso_do_solo['uso_pred_simples_num'].isin(uso_pred_simples)]
    
    cruzamentos =  gpd.sjoin(df_estacoes_circulo, df_uso_do_solo_filtrado, how="inner", predicate="intersects")
    
    cruzamentos = cruzamentos.merge(
        df_uso_do_solo[["Name", "geometry"]],  # Apenas as colunas relevantes
        on="Name",  # A coluna para equivalência
        how="left"  # Tipo de junção (left mantém todos os dados de cruzamentos)
    )
    ocorrencias_necessarias_para_plot = cruzamentos.copy()
  
    # layer_uso_do_solo =  folium.FeatureGroup(name = "Uso do Solo")
    layer_estacoes = folium.FeatureGroup(name = "Estações")
    layer_circulos_estacoes = folium.FeatureGroup(name = f"Redondezas {radius_km} km") 
    
    ## Plot do uso do solo
    layers_dict = {}
    for place in uso_pred_simples:
        ocorrencia_por_plot = ocorrencias_necessarias_para_plot[ocorrencias_necessarias_para_plot['uso_pred_simples_num'] == place]
        layer_uso_do_solo = folium.FeatureGroup(
        name=str(ocorrencias_necessarias_para_plot['uso_pred_simples']
             [ocorrencias_necessarias_para_plot['uso_pred_simples_num'] == place].values[0]),
        overlay = True) 
        layers_dict[place] = layer_uso_do_solo
        for _, row in ocorrencia_por_plot.iterrows():
            cor = get_color(row['uso_pred_simples_num'])
            geo_json = row['geometry'].__geo_interface__
            folium.GeoJson(
                geo_json, 
                tooltip = str(row['Name']) + " Classe " + str(row['uso_pred_simples']),
                style_function = lambda x, color = cor:{'fillColor' : color , 'color' : color, 'weight' : 1}
            ).add_to(layers_dict[place])
        layers_dict[place].add_to(mapa)
        # layers_dict[place].control = False
    # folium.LayerControl(collapsed=False).add_to(mapa)
    # return mapa

    cruzamentos = cruzamentos.groupby('name', as_index = False)['uso_pred_simples_num'].count()
    cruzamentos= pd.merge(cruzamentos, df_estacoes_circulo, on = 'name',  how = 'left')

    # scaler = MinMaxScaler()
    # cruzamentos['uso_pred_simples_num'] = scaler.fit_transform(cruzamentos[['uso_pred_simples_num']])
    out_lower, out_upper, non_outliers = identificar_outliers(cruzamentos, "uso_pred_simples_num")
    colormap = cm.linear.plasma.scale(non_outliers['uso_pred_simples_num'].min(), 
                                      non_outliers['uso_pred_simples_num'].max())
    colormap.add_to(mapa)
    
    plot_circlemarkers_outliers(out_lower, out_upper, non_outliers, "black", "green", 5, 
                                "name", "uso_pred_simples_num", layer_estacoes, colormap)  

    ## Plota círculos em volta das estações
    for _, row in df_estacoes_circulo.iterrows():    
        folium.GeoJson(
            row['circle'].__geo_interface__, 
            tooltip = row['name'],
            style_function=lambda x:{'fillColor': 'gray', 'color': 'gray', 'fillOpacity': 0.0},
        ).add_to(layer_circulos_estacoes)
        
    # layer_uso_do_solo.add_to(mapa)
    layer_estacoes.add_to(mapa)
    layer_circulos_estacoes.add_to(mapa)
    folium.LayerControl().add_to(mapa)
    return mapa

In [ ]:
uso_do_solo = read_kmz_directly("SIRGAS_KMZ_uso_predominante_simples_2021.kmz")
estacoes = pd.read_parquet("estacoes_reestruturadas.parquet")

In [ ]:
df_uso_do_solo = tratar_df_uso_do_solo(uso_do_solo)
df_estacoes = tratar_df_estacoes(estacoes)

In [ ]:
analise_uso_do_solo(df_uso_do_solo, df_estacoes, 1, [1,3,4,6,9,12,15])

# Resumo = {
#         "Residencial Horizontal Médio Padrão": 1,
#         "Residencial Horizontal Baixo Padrão": 2,
#         "Comércio e Serviço Horizontal": 3,
#         "Residencial Vertical Médio Padrão": 4,
#         "Residencial Horizontal Alto Padrão": 5,
#         "Residencial Vertical Alto Padrão": 6,
#         "Comércio e Serviço Vertical": 7,
#         "Residencial Vertical Baixo Padrão": 8,
#         "Terrenos Vagos": 9,
#         "Indústrias": 10,
#         "Escolas": 11,
#         "Usos Coletivos": 12,
#         "Usos Especiais": 13,
#         "Armazéns e Depósitos": 14,
#         "Sem correspondência": 15,
#         "Outros Usos": 16,
#         "Garagens": 17
#     }

In [ ]:
df_uso_do_solo.columns

In [ ]:
df_uso_do_solo.head()

In [ ]:
df_uso_do_solo['uso_pred_simples_num'].value_counts()

In [ ]:
# df_uso_do_solo['uso_pred_60'].value_counts()
df_uso_do_solo.shape